In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import image_dataset_from_directory

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
dataset_config = {
    'labels'            : 'inferred',
    'label_mode'        : 'categorical',
    'class_names'       : ['Russula', 'Entoloma', 'Amanita', 'Lactarius', 'Cortinarius', 'Hygrocybe', 'Agaricus', 'Suillus', 'Boletus'],
    'color_mode'        : 'grayscale',
    'batch_size'        : 64,
    'shuffle'           : True,
    'seed'              : 42,
    'validation_split'  : 0.25,
    'image_size'        : (256, 256),
    'interpolation'     : 'bilinear',
    'follow_links'      : False
    }

train_dataset = image_dataset_from_directory(
    'train',
    labels            = dataset_config['labels'],
    label_mode        = dataset_config['label_mode'],
    class_names       = dataset_config['class_names'],
    color_mode        = dataset_config['color_mode'],
    batch_size        = dataset_config['batch_size'],
    image_size        = dataset_config['image_size'],
    shuffle           = dataset_config['shuffle'],
    seed              = dataset_config['seed'],
    validation_split  = dataset_config['validation_split'],
    subset            = 'training',
    interpolation     = dataset_config['interpolation'],
    follow_links      = dataset_config['follow_links'],
)

test_dataset = image_dataset_from_directory(
    'test',
    labels            = dataset_config['labels'],
    label_mode        = dataset_config['label_mode'],
    class_names       = dataset_config['class_names'],
    color_mode        = dataset_config['color_mode'],
    batch_size        = dataset_config['batch_size'],
    image_size        = dataset_config['image_size'],
    shuffle           = dataset_config['shuffle'],
    seed              = dataset_config['seed'],
    subset            = None,
    interpolation     = dataset_config['interpolation'],
    follow_links      = dataset_config['follow_links'],
)

val_dataset = image_dataset_from_directory(
    'train',
    labels            = dataset_config['labels'],
    label_mode        = dataset_config['label_mode'],
    class_names       = dataset_config['class_names'],
    color_mode        = dataset_config['color_mode'],
    batch_size        = dataset_config['batch_size'],
    image_size        = dataset_config['image_size'],
    shuffle           = dataset_config['shuffle'],
    seed              = dataset_config['seed'],
    validation_split  = dataset_config['validation_split'],
    subset            = 'validation',
    interpolation     = dataset_config['interpolation'],
    follow_links      = dataset_config['follow_links'],
)

In [ ]:
# https://blogs.oracle.com/meena/simple-neural-network-model-using-keras-and-grid-search-hyperparameterstuning
activation = ['relu','tanh','sigmoid','linear']
momentum = [0.0, 0.2, 0.5, 0.6, 0.8, 0.9]
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
weight_constraint = [1, 2, 3, 4, 5]
epochs = [1, 10, 20, 25, 50, 100, 150]
batch_size = [8, 16, 32, 64, 128]
param_grid = {
    'activation'        : activation,
    'momentum'          : momentum,
    'learn_rate'        : learn_rate,
    'weight_constraint' : weight_constraint,
    'epochs'            : epochs,
    'batch_size'        : batch_size,
}

In [ ]:
# Create the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(dataset_config['image_size'][0], dataset_config['image_size'][1], 1), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(9))
model.add(Activation('softmax'))

epochs = 50
optimizer = 'Adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_dataset, validation_data = val_dataset, epochs = epochs, batch_size = 64)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_dataset, verbose = 0)
print("Accuracy: %.2f%%" % (scores[1] * 100))